In [1]:
from typing import List, Dict, Set, Any, Optional, Tuple, Literal, Callable
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from tqdm import tqdm
import numpy as np
import torch
from torch import Tensor
from kernels.static_kernels import LinearKernel, RBFKernel
from kernels.sig_trunc import TruncSigKernel

from features.random_fourier import RBF_RandomFourierFeatures
from features.random_sig_fourier import TRP_RFSF_Gaussian, TRP_RFSF_Linear
from features.random_warping_series import RandomWarpingSeries
from features.signature import sig, logsig

In [ ]:
# #######################
# ######### RWS #########
# #######################

def RWS_test():
    # Setup and parameters
    N = 3
    N2 = 2
    T = 70
    d = 2
    dtype = torch.float32
    #torch.manual_seed(3)
    X = torch.randn(N, T, d, dtype=dtype).to("cuda").detach()
    Y = torch.randn(N2,T, d, dtype=dtype).to("cuda").detach()
    n_features = 20
    D_min=2
    D_max=5
    sigma=1

    # RWS
    rws = RandomWarpingSeries(n_features, D_min, D_max, sigma, LinearKernel(scale=1/d/T))
    feat_X = rws(X)
    feat_Y = rws(Y)
    K_rws = 1 + feat_X @ feat_Y.T
    print(feat_X)
    print(feat_X.shape)
    print(K_rws)

RWS_test()

In [10]:
###############################
#### Lib signatory wrapper ####
###############################

def test_signatory():
    N,T,d = 1, 30, 2
    trunc_level = 3
    X = torch.randn(N, T, d)
    sigs = sig(X, trunc_level) # potential unknown bug. Says imput is not 3D, but it clearly is.
    logsigs = logsig(X, trunc_level)
    print("sigs", sigs.shape, sigs)
    print("logsigs", logsigs.shape, logsigs)

# test_signatory()

In [11]:
##########################################################
#### Linear TRP-RFSF features  vs  Vanilla Sig kernel ####
##########################################################

def LINEAR_trp_vs_kernel():
    #parameters
    N = 3
    N2 = 2
    T = 20
    d = 2
    trunc_level = 5
    n_features = 100
    dtype = torch.float32
    #torch.manual_seed(3)
    X = torch.randn(N, T, d, dtype=dtype).to("cuda").detach() / np.sqrt(d)
    Y = torch.randn(N2,T, d, dtype=dtype).to("cuda").detach() / np.sqrt(d)

    #exact sig kernel
    sigker = TruncSigKernel(LinearKernel(), normalize=False, trunc_level=trunc_level, geo_order=1, max_batch=50000)
    K = sigker(X,Y)
    print("exact\n", K)

    #trp
    MC_iter = 10000
    res = []
    for i in tqdm(range(MC_iter)):
        trp = TRP_RFSF_Linear(trunc_level, n_features, only_last=True)
        feat_X = trp(X)
        feat_Y = trp(Y)
        K_trp = 1 + feat_X @ feat_Y.T
        res.append(K_trp)
    res = torch.stack(res)
    example = res[0]
    mean = res.mean(dim=0)
    print("mean\n", mean)
    print("example\n", example)

LINEAR_trp_vs_kernel()
# exact
#  tensor([[ 4627.0444, -5744.9629],
#         [ 9709.9150, -7802.3965],
#         [-1727.5579,  2563.8979]], device='cuda:0')
# 100%|██████████| 10000/10000 [00:15<00:00, 641.01it/s] # n_features = 500, trunc_level = 5
# mean
#  tensor([[ 4624.3896, -5737.0781],
#         [ 9733.1250, -7796.5479],
#         [-1733.9088,  2575.4778]], device='cuda:0')

exact
 tensor([[  1878.4482,    -74.9327],
        [-31913.9102,   3453.7073],
        [ -3055.4370,   -211.4108]], device='cuda:0')


100%|██████████| 10000/10000 [00:15<00:00, 649.39it/s]

mean
 tensor([[  1921.0502,    -57.7952],
        [-31457.2656,   3511.3784],
        [ -3003.9236,   -212.3938]], device='cuda:0')
example
 tensor([[   -85.2137,   -168.0927],
        [-10564.7783,   6568.4390],
        [ -1389.7279,   -176.8557]], device='cuda:0')


In [13]:
#######################################################
#### Gaussian TRP-RFSF features  vs  SigRBF kernel ####
#######################################################

def GAUSSIAN_trp_vs_kernel():
    #parameters
    N = 3
    N2 = 2
    T = 20
    d = 2
    trunc_level = 5
    n_features = 5000
    sigma = 1.0
    dtype = torch.float32
    #torch.manual_seed(3)
    X = torch.randn(N, T, d, dtype=dtype).to("cuda").detach() / np.sqrt(d)
    Y = torch.randn(N2,T, d, dtype=dtype).to("cuda").detach() / np.sqrt(d)

    #exact sig kernel
    sigker = TruncSigKernel(RBFKernel(sigma=sigma), normalize=False, trunc_level=trunc_level, geo_order=1, max_batch=50000)
    K = sigker(X,Y)
    print("exact\n", K)

    #trp
    MC_iter = 10000
    res = []
    for i in tqdm(range(MC_iter)):
        trp = TRP_RFSF_Gaussian(trunc_level, n_features, sigma, only_last=True)
        feat_X = trp(X)
        feat_Y = trp(Y)
        K_trp = 1 + feat_X @ feat_Y.T
        res.append(K_trp)
    res = torch.stack(res)
    example = res[0]
    mean = res.mean(dim=0)
    print("mean\n", mean)
    print("example\n", example)

GAUSSIAN_trp_vs_kernel()
# GAUSSIAN_trp_vs_kernel()
# # exact
# #  tensor([[ -8.7953, -30.1428],
# #         [ -4.1622,  -4.5515],
# #         [-25.4650,  42.8830]], device='cuda:0')
# # 100%|██████████| 10000/10000 [05:55<00:00, 28.10it/s] # n_features=5000, trunc_level=5
# # mean
# #  tensor([[ -9.9236, -30.5771],
# #         [ -4.4291,  -4.5940],
# #         [-25.3243,  43.3625]], device='cuda:0')

exact
 tensor([[  2.9611, -27.3819],
        [-11.7590, -37.3600],
        [ -1.9176,  -6.2990]], device='cuda:0')


100%|██████████| 10000/10000 [07:07<00:00, 23.39it/s]

mean
 tensor([[  2.9587, -27.4341],
        [-11.7393, -37.0513],
        [ -1.8954,  -6.3103]], device='cuda:0')
example
 tensor([[-10.4052, -16.3350],
        [-15.8057, -42.5761],
        [-25.0634,  -8.4635]], device='cuda:0')


In [14]:
############################################################
#### Test RBF_RandomFourierFeatures vs exact RBF kernel ####
############################################################
def rff_vs_exact_RBFKernel():
    N=3
    N2= 2
    d = 10
    sigma=1
    dtype = torch.float64
    # torch.manual_seed(1)
    X = torch.randn(N, d, dtype=dtype).to("cuda") /np.sqrt(d)
    Y = torch.randn(N2, d, dtype=dtype).to("cuda") / np.sqrt(d)

    # Exact RBF kernel
    k = RBFKernel(sigma=sigma)
    K = k(X, Y)

    # Approximate RBF kernel using RBF_RandomFourierFeatures
    N_MC = 10000
    res = []
    for i in range(N_MC):
        RFF = RBF_RandomFourierFeatures(n_features=1000,
                                        sigma=sigma,
                                        method="cos(x)sin(x)",
                                        # method = "cos(x + b)",
                                        )
        feat_X = RFF(X)
        feat_Y = RFF(Y)
        K_rff = feat_X @ feat_Y.T
        res.append(K_rff)
    K_rff = torch.mean(torch.stack(res), dim=0)

    print("K\n",K)
    print("K_rff\n",K_rff)
    print("diff\n", K-K_rff)
    print("diffmean\n", torch.mean(abs(K-K_rff)))
    # the RFF approach cant reproduce results smaller than 1e-5 for some reason
    
rff_vs_exact_RBFKernel()

K
 tensor([[0.7222, 0.3840],
        [0.2072, 0.4892],
        [0.7078, 0.2584]], device='cuda:0', dtype=torch.float64)
K_rff
 tensor([[0.7221, 0.3844],
        [0.2069, 0.4893],
        [0.7078, 0.2586]], device='cuda:0', dtype=torch.float64)
diff
 tensor([[ 1.6111e-04, -3.7687e-04],
        [ 2.6111e-04, -4.7682e-05],
        [-5.2112e-05, -1.3948e-04]], device='cuda:0', dtype=torch.float64)
diffmean
 tensor(0.0002, device='cuda:0', dtype=torch.float64)
